In [2]:
import json
import pandas as pd
import copy
import numpy as np
from scipy.signal import correlate

python(14890) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


## Data preprocessing

#### Open file

In [3]:
with open('./ASK_flavell/2022-08-02-01.json', 'r') as file:
    data = json.load(file)

with open('./ASK_flavell/2023-01-19-22.json', 'r') as file2:
    data_2023 = json.load(file2)

In [4]:
data.keys() #inspect data structure

dict_keys(['avg_timestep', 'rel_enc_str_θh', 'trace_original', 'dorsalness', 'head_curvature', 'trace_array', 'angular_velocity', 'rel_enc_str_P', 'reversal_events', 'encoding_changing_neurons', 'feedingness', 'uid', 'ranges', 'labeled', 'timestamp_confocal', 'velocity', 'body_curvature', 'forwardness', 'tau_vals', 'num_neurons', 'rel_enc_str_v', 'max_t', 'dataset_type', 'pumping', 'neuron_categorization'])

In [5]:
len(data['labeled'])

115

In [7]:
data['avg_timestep']

0.010025914985390573

In [8]:
def flavell_data_to_df(data, specific_col, neuron_label_col, label_name):
    """
    For time series data like flavell 2023, convert from original JSON type to pandas dataframe
    Returns simplified data in pandas type with only specific column of interest listed with neuron labels
    """

# Find matching labels
    simplified_label_dict = {key: value[label_name] for key, value in data[neuron_label_col].items()}
    for i in range(1,153):
        # print(i)
        if str(i) in simplified_label_dict.keys():
            continue
        else:
            # print(i,'not found')
            simplified_label_dict[str(i)]= 'unknown'
    # print(data['trace_array'][0]) #inspect data
    # Extract the relevant data
    neuron_label = data[neuron_label_col]
    specific_array = data[specific_col]

    # Convert to DataFrame
    df = pd.DataFrame(specific_array).T
    df.columns = [str(i+1) for i in range(len(df.columns))]

    # Rename column names with corresponding neuron labels
    df.rename(columns=simplified_label_dict, inplace=True)
    df = df.drop(columns=[col for col in df.columns if col == 'unknown'])
    return df
# df.head()

In [9]:
flavell_2023 = flavell_data_to_df(data_2023,'trace_array','labeled','label')

In [10]:
# Extract data, convert to corresponding dataframe with matching neuron labels

# Find matching labels
simplified_label_dict = {key: value['label'] for key, value in data_2023['labeled'].items()}
for i in range(1,153):
    # print(i)
    if str(i) in simplified_label_dict.keys():
        continue
    else:
        # print(i,'not found')
        simplified_label_dict[str(i)]= 'unknown'
# print(data_2023['trace_array'][0]) #inspect data
# Extract the relevant data
neuron_label = data_2023['labeled']
trace_array = data_2023['trace_array']

# Convert to DataFrame
df_2023 = pd.DataFrame(trace_array).T
df_2023.columns = [str(i+1) for i in range(len(df_2023.columns))]

# Rename column names with corresponding neuron labels
df_2023.rename(columns=simplified_label_dict, inplace=True)
df_2023.head()

,unknown,ASGL,AVER,unknown,RMER,OLLL,RMDDR,unknown,RMDVR,unknown,...,RIR,RIBL,RMDDL,AIY?,SAADR,unknown,unknown,OLQDR,IL2VL,IL1DL
0,1.718261,2.246925,1.527276,-0.486978,0.692619,1.304705,0.749223,-1.099329,1.262640,2.245194,...,0.564302,-0.083529,1.734464,3.021633,1.391277,3.156553,0.967802,3.716949,-0.432095,0.524865
1,1.187032,2.145634,1.194294,-0.340825,1.296790,1.203598,1.754050,-1.144748,0.824632,2.359974,...,0.494005,-0.213839,1.146327,3.079677,1.262743,2.497042,0.502968,2.148008,-0.443719,0.414394
2,0.707739,2.901741,0.808551,-0.839290,0.919021,1.859613,1.879834,-1.209404,0.413063,1.996590,...,0.745321,-0.168911,0.255168,2.779931,0.823848,2.710983,0.338570,2.119922,-0.426385,1.077881
3,0.313456,2.725000,0.785007,-0.978039,0.537547,2.230192,0.059686,-1.282246,0.066860,2.330346,...,1.169363,-0.110392,-0.116520,2.540096,-0.275289,1.947485,0.281470,1.928530,-0.148742,1.722164
4,0.113525,2.430254,0.574354,-1.318454,0.383169,2.239677,-0.204969,-1.191290,0.043417,2.204133,...,1.475376,-0.162964,0.357824,2.343885,-0.682820,1.877508,0.224883,1.708467,-0.358856,1.834026


In [11]:
# Drop traces with unknown neuron identity 
df_2023 = df_2023.drop(columns=[col for col in df_2023.columns if col == 'unknown'])

In [13]:
# Inspect data
print(df_2023.shape)
df_2023.head()


(1600, 96)


,ASGL,AVER,RMER,OLLL,RMDDR,RMDVR,AIZL,RMGL,AIML,IL1VR,...,AIBR,ASKR,RIR,RIBL,RMDDL,AIY?,SAADR,OLQDR,IL2VL,IL1DL
0,2.246925,1.527276,0.692619,1.304705,0.749223,1.262640,0.727764,-0.382538,1.453225,-0.009720,...,0.865732,1.035116,0.564302,-0.083529,1.734464,3.021633,1.391277,3.716949,-0.432095,0.524865
1,2.145634,1.194294,1.296790,1.203598,1.754050,0.824632,0.938731,-0.506756,1.601682,-0.036858,...,0.914127,1.688427,0.494005,-0.213839,1.146327,3.079677,1.262743,2.148008,-0.443719,0.414394
2,2.901741,0.808551,0.919021,1.859613,1.879834,0.413063,0.910760,-0.445583,2.410355,-0.121467,...,0.480493,1.759862,0.745321,-0.168911,0.255168,2.779931,0.823848,2.119922,-0.426385,1.077881
3,2.725000,0.785007,0.537547,2.230192,0.059686,0.066860,0.754841,-0.683376,2.746414,-0.221022,...,0.198858,1.820026,1.169363,-0.110392,-0.116520,2.540096,-0.275289,1.928530,-0.148742,1.722164
4,2.430254,0.574354,0.383169,2.239677,-0.204969,0.043417,0.688548,-0.537424,2.668919,-0.829297,...,0.079380,1.554967,1.475376,-0.162964,0.357824,2.343885,-0.682820,1.708467,-0.358856,1.834026


In [21]:
def create_timetrace(avg_step, length):
    """
    Create timetrace based on the average timestep recorded for each day's recording
    Args:
    avg_step: average timestep in data['avg_timestep']
    length: total steps needed
    """
    timetrace = np.zeros(length)
    for i in range(1,length):
        timetrace[i] = timetrace[i-1] + avg_step
    return timetrace


In [22]:
timetrace = create_timetrace(data['avg_timestep'],1600)
# df['Timetrace']=timetrace


## Correlation analysis

### Whole Activitye Trace (considering time lags)

In [23]:
df_2023.head()

,ASGL,AVER,RMER,OLLL,RMDDR,RMDVR,AIZL,RMGL,AIML,IL1VR,...,AIBR,ASKR,RIR,RIBL,RMDDL,AIY?,SAADR,OLQDR,IL2VL,IL1DL
0,2.246925,1.527276,0.692619,1.304705,0.749223,1.262640,0.727764,-0.382538,1.453225,-0.009720,...,0.865732,1.035116,0.564302,-0.083529,1.734464,3.021633,1.391277,3.716949,-0.432095,0.524865
1,2.145634,1.194294,1.296790,1.203598,1.754050,0.824632,0.938731,-0.506756,1.601682,-0.036858,...,0.914127,1.688427,0.494005,-0.213839,1.146327,3.079677,1.262743,2.148008,-0.443719,0.414394
2,2.901741,0.808551,0.919021,1.859613,1.879834,0.413063,0.910760,-0.445583,2.410355,-0.121467,...,0.480493,1.759862,0.745321,-0.168911,0.255168,2.779931,0.823848,2.119922,-0.426385,1.077881
3,2.725000,0.785007,0.537547,2.230192,0.059686,0.066860,0.754841,-0.683376,2.746414,-0.221022,...,0.198858,1.820026,1.169363,-0.110392,-0.116520,2.540096,-0.275289,1.928530,-0.148742,1.722164
4,2.430254,0.574354,0.383169,2.239677,-0.204969,0.043417,0.688548,-0.537424,2.668919,-0.829297,...,0.079380,1.554967,1.475376,-0.162964,0.357824,2.343885,-0.682820,1.708467,-0.358856,1.834026


In [24]:
list(df_2023['IL1R'])[1599]

-0.7509630758022752

In [25]:
import numpy as np

def time_lagged_correlation(series1, series2, max_lag=10):
    """
    Compute cross-correlation between two 1D numeric series for a range of time lags.
    Returns a dict of lag -> correlation matrix from np.corrcoef.
    """
    # Convert to numpy arrays
    series1 = np.asarray(series1).ravel()
    series2 = np.asarray(series2).ravel()
    
    # Truncate to same length
    cut = min(len(series1), len(series2))
    series1 = series1[:cut]
    series2 = series2[:cut]
    assert len(series1) == len(series2)
    
    correlations = {}
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            shifted_series1 = series1[-lag:]   # e.g. lag=-5 -> shift forward by 5
            shifted_series2 = series2[:lag]    # up to len(series2) - 5
        elif lag > 0:
            shifted_series1 = series1[:-lag]   # up to len(series1) - lag
            shifted_series2 = series2[lag:]    # from lag to end
        else:
            shifted_series1 = series1
            shifted_series2 = series2
        
        # Print shape debug if needed
        # print(f"lag={lag}, s1={shifted_series1.shape}, s2={shifted_series2.shape}")
        
        # They must be the same length to correlate
        if len(shifted_series1) != len(shifted_series2) or len(shifted_series1) == 0:
            # skip or set correlation to NaN
            correlations[lag] = np.nan
        else:
            correlations[lag] = np.corrcoef(shifted_series1, shifted_series2)
    
    return correlations


def find_max_correlations(results, top_n=None, use_abs=True):
    """
    Given a dictionary of {(neuron1, neuron2): {lag: corr_matrix}}, 
    find the highest correlation across lags for each neuron pair and return a list.
    
    :param results: dict 
        Keys are (neuron1, neuron2), values are {lag: 2x2 corr matrix from np.corrcoef}.
    :param top_n: int or None
        If given, return only the top_n results by correlation strength.
    :param use_abs: bool
        If True, rank correlations by absolute value; otherwise, use raw correlation.
    :return: List of dicts, each with keys:
             {
                'pair': (neuron1, neuron2),
                'lag': best_lag,
                'corr': correlation_coefficient
             }
             Sorted in descending order of correlation strength.
    """
    summary = []

    for pair, lag_dict in results.items():
        best_corr = None
        best_lag = None
        
        # Iterate over each lag, pick the correlation coefficient
        for lag, corr_matrix in lag_dict.items():
            # Some entries might be NaN if there's a mismatch in time steps
            if isinstance(corr_matrix, float) and np.isnan(corr_matrix):
                # skip if we stored np.nan
                continue
            
            # corr_matrix is 2x2 from np.corrcoef(series1, series2)
            # The off-diagonal elements [0,1] or [1,0] have the correlation
            corr_val = corr_matrix[0, 1]

            # Decide if we compare absolute or raw correlation
            compare_val = abs(corr_val) if use_abs else corr_val

            # Track the maximum correlation
            if best_corr is None or compare_val > (abs(best_corr) if use_abs else best_corr):
                best_corr = corr_val
                best_lag = lag
        
        # Once we find the best correlation + lag for this pair, add to summary
        if best_corr is not None:
            summary.append({
                "pair": pair,
                "lag": best_lag,
                "corr": best_corr
            })

    # Sort the summary list by correlation strength (descending)
    # If using absolute correlation, sort by abs(corr); otherwise sort by raw corr
    if use_abs:
        summary.sort(key=lambda x: abs(x["corr"]), reverse=True)
    else:
        summary.sort(key=lambda x: x["corr"], reverse=True)

    # If user provided top_n, slice
    if top_n is not None:
        summary = summary[:top_n]

    return summary


In [ ]:
# Compute time-lagged correlations for all pairs of neurons!
max_lag = 5  # Maximum time lag to consider
results = {}

for neuron1 in df_2023.columns:
    for neuron2 in df_2023.columns:
        if neuron1 != neuron2:
            # print(neuron1, neuron2)
            correlations = time_lagged_correlation(df_2023[neuron1], df_2023[neuron2], max_lag)
            results[(neuron1, neuron2)] = correlations

In [27]:
# Suppose you've already computed 'results' from your time-lagged correlation function
ranked_results = find_max_correlations(results, top_n=1000, use_abs=True)

# for entry in top_results:
#     pair = entry["pair"]       # e.g. ('AIBL', 'AIZL')
#     lag = entry["lag"]         # e.g. -2
#     corr = entry["corr"]       # e.g. 0.85
#     print(f"Neuron pair {pair}, best lag={lag}, correlation={corr:.3f}")
ranked_results = pd.DataFrame(ranked_results)
ranked_results

,pair,lag,corr
0,"(AIBL, AIBR)",0,0.965740
1,"(AIBR, AIBL)",0,0.965740
2,"(AVAR, AVAL)",1,0.954173
3,"(AVAL, AVAR)",-1,0.954173
4,"(AVER, AVEL)",0,0.952468
...,...,...,...
995,"(RIR, AUAR)",2,0.473003
996,"(AUAL, CEPVL)",-2,0.472862
997,"(CEPVL, AUAL)",2,0.472862
998,"(CEPDL, URYVL)",-5,0.472832


In [ ]:
ranked_results.to_excel('/Users/yufeimeng/Desktop/SP25/Thesis/final_corr_results_2023.xlsx', index=False)

## Mutual Information metrics

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mutual_info_score

def mutual_info_discrete(x, y, n_bins=10):
    """
    Compute mutual information between two continuous variables by discretizing into bins.
    """
    # Discretize both arrays into the same bins
    bins = np.histogram_bin_edges(np.concatenate([x, y]), bins=n_bins)
    x_disc = np.digitize(x, bins) - 1
    y_disc = np.digitize(y, bins) - 1
    return mutual_info_score(x_disc, y_disc)

def mutual_info_continuous(x, y, random_state=None):
    """
    Estimate mutual information between two continuous variables using k-NN estimator.
    """
    # Reshape inputs for sklearn
    x_arr = x.values.reshape(-1, 1) if hasattr(x, "values") else np.array(x).reshape(-1, 1)
    y_arr = y.values if hasattr(y, "values") else np.array(y)
    return mutual_info_regression(x_arr, y_arr, discrete_features=False, random_state=random_state)[0]

def compute_pairwise_mi(df, method="continuous", bins=10, random_state=None):
    """
    Compute mutual information for all unique neuron pairs in DataFrame df.
    :param df: DataFrame with shape (time_points, neurons)
    :param method: "continuous" (k-NN) or "discrete" (histogram binning)
    :param bins: number of bins for discrete method
    :param random_state: seed for reproducibility in continuous method
    :return: DataFrame with columns ['neuron1', 'neuron2', 'MI'] sorted by descending MI
    """
    results = []
    neurons = df.columns
    for i in range(len(neurons)):
        for j in range(i + 1, len(neurons)):
            n1, n2 = neurons[i], neurons[j]
            x, y = df[n1], df[n2]
            if method == "continuous":
                mi = mutual_info_continuous(x, y, random_state=random_state)
            else:
                mi = mutual_info_discrete(x, y, n_bins=bins)
            results.append({"neuron1": n1, "neuron2": n2, "MI": mi})
    mi_df = pd.DataFrame(results)
    return mi_df.sort_values("MI", ascending=False)

In [ ]:
# Compute MI!
df_mi = compute_pairwise_mi(df_2023, method="continuous", random_state=0)

In [ ]:
df_mi.to_excel("/Users/yufeimeng/Desktop/SP25/Thesis/mutual_info_ranked_final_2023.xlsx", index=False)

## Transfer Entropy implementation

In [45]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.special import digamma

def transfer_entropy_knn(source, target, lag=1, k=5):
    """
    Estimate transfer entropy TE_{source -> target} using k-NN (KSG) estimator:
      TE = I(X_past; Y_present | Y_past)
    Args:
      source, target: 1D arrays or pandas Series of equal length
      lag: time lag (in samples)
      k: number of nearest neighbors
    Returns:
      TE estimate in bits
    """
    x = np.asarray(source).ravel()
    y = np.asarray(target).ravel()
    N = len(x)
    if abs(lag) < 1:
        raise ValueError("lag absolute value must be greater than 1")
    # Build variables
    x_p = x[:-lag].reshape(-1, 1)
    y_p = y[:-lag].reshape(-1, 1)
    y_c = y[lag:].reshape(-1, 1)
    # Joint and marginal spaces
    xyz = np.hstack([x_p, y_p, y_c])
    xz  = np.hstack([x_p, y_p])
    yz  = np.hstack([y_p, y_c])
    z   = y_p
    # Fit k-NN on joint space
    nbrs = NearestNeighbors(n_neighbors=k+1, metric='chebyshev')
    nbrs.fit(xyz)
    distances, _ = nbrs.kneighbors(xyz)
    # k+1 because the first neighbor is itself (distance=0)
    eps = distances[:, k] + 1e-16  # use the k-th neighbor distance
    # Count neighbors in each subspace within eps
    def count_neighbors(data, eps):
        nbr = NearestNeighbors(metric='chebyshev')
        nbr.fit(data)
        # count neighbors strictly within eps (exclude the point itself)
        counts = nbr.radius_neighbors(data, radius=eps, return_distance=False)
        return np.array([len(c) - 1 for c in counts])  # subtract self
    nxz = count_neighbors(xz, eps)
    nyz = count_neighbors(yz, eps)
    nz  = count_neighbors(z, eps)
    # KSG transfer entropy: TE = psi(k) + mean[psi(nz+1) - psi(nxz+1) - psi(nyz+1)]
    te = digamma(k) + np.mean(digamma(nz + 1) - digamma(nxz + 1) - digamma(nyz + 1))
    # Convert from nats to bits
    return te / np.log(2)

def compute_transfer_entropy_knn_matrix(df, lag=1, k=5):
    """
    Compute transfer entropy for all ordered neuron pairs in DataFrame df.
    Returns a DataFrame sorted by descending TE.
    """
    results = []
    neurons = df.columns.tolist()
    for src in neurons:
        for tgt in neurons:
            if src == tgt:
                continue
            te_val = transfer_entropy_knn(df[src], df[tgt], lag=lag, k=k)
            results.append({"source": src, "target": tgt, "TE": te_val})
    te_df = pd.DataFrame(results)
    return te_df.sort_values("TE", ascending=False).reset_index(drop=True)

In [48]:
#compute KSG transfer entropy for lags from -5 to 5 and save ranked results

k = 5
lags = range(-5, 6)  # -5, -4, ..., 0, 1, ..., 5

all_results = []

for lag in lags:
    if lag == 0:
        continue #avoid having 0 lag
    else:
        print(f"computing lag {lag}")
        for src in df_2023.columns:
            for tgt in df_2023.columns:
                if src == tgt:
                    continue
                if lag > 0:
                    te_val = transfer_entropy_knn(df_2023[src], df_2023[tgt], lag=lag, k=k)
                else:
                    # for negative lag, swap roles (equivalent to TE_{tgt->src} at positive lag)
                    te_val = transfer_entropy_knn(df_2023[tgt], df_2023[src], lag=-lag, k=k)
                all_results.append({
                    "lag": lag,
                    "source": src,
                    "target": tgt,
                    "TE": te_val
                })
        print("finished calculation!")


computing lag -5
finished calculation!
computing lag -4
finished calculation!
computing lag -3
finished calculation!
computing lag -2
finished calculation!
computing lag -1
finished calculation!
computing lag 1
finished calculation!
computing lag 2
finished calculation!
computing lag 3
finished calculation!
computing lag 4
finished calculation!
computing lag 5
finished calculation!


In [50]:
te_all_df = pd.DataFrame(all_results)
te_all_df = te_all_df.sort_values("TE", ascending=False).reset_index(drop=True)
te_all_df.to_excel("/Users/yufeimeng/Desktop/SP25/Thesis/transfer_entropy_knn_lagged.xlsx", index=False)

In [1]:
import pandas as pd

# Transcribed values from the screenshot
values = [-0.19, -0.38, -0.21, -0.41, -0.40, -0.63, -0.48, -0.48, -0.74, -0.61, -0.79]

# Create DataFrame and save as CSV
df = pd.DataFrame(values, columns=['value'])
df['value'].mean()


np.float64(-0.48363636363636364)